### 통합 - 숙소 상세 정보, 방 상세정보, 가격 정보 크롤링 데이터 수정

방, 가격 테이블에서 accommodationId 가 숙소 테이블의 ano를 받아왔던 것을 숙소 테이블에 id 컬럼을 추가하여 ano -> id 값으로 변경

In [117]:
import requests
import json
import pandas as pd
from tqdm import tqdm
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import re

숙소 상세정보에 id 추가

In [118]:
acc_detail = pd.read_csv(f'../data/여기어때_숙소상세정보전체.csv')
acc_detail.head()

,Unnamed: 0,id,category,name,score,region,ano,lat,lng,pic_url,address,pic_urls,detail
0,0,0,1,강남 더 뮤즈,9.8,강남구 역삼동,1824,37.502530,127.038275,https://image.goodchoice.kr/resize_1000X500x0/...,서울 강남구 역삼동 678-9,['https://image.goodchoice.kr/resize_490x348/a...,"<article class=""detail_info"">\n<!-- 기본 정보 -->\..."
1,1,1,1,역삼 호텔 디 아티스트,9.3,강남구 역삼동,5675,37.502415,127.038488,https://image.goodchoice.kr/resize_1000X500x0/...,서울 강남구 역삼동 678-13,['https://image.goodchoice.kr/resize_490x348/a...,"<article class=""detail_info"">\n<!-- 기본 정보 -->\..."
2,2,2,1,역삼 리치웰,9.3,강남구 역삼동,3805,37.502973,127.039823,https://image.goodchoice.kr/resize_1000X500x0/...,서울 강남구 역삼동 677-9,['https://image.goodchoice.kr/resize_490x348/a...,"<article class=""detail_info"">\n<!-- 기본 정보 -->\..."
3,3,3,1,강남 캠퍼스,9.3,강남구 역삼동,4988,37.497310,127.029190,https://image.goodchoice.kr/resize_1000X500x0/...,서울 강남구 역삼동 825-30,['https://image.goodchoice.kr/resize_490x348/a...,"<article class=""detail_info"">\n<!-- 기본 정보 -->\..."
4,4,4,1,역삼 인트로호텔,9.9,강남구 역삼동,72748,37.502754,127.039923,https://image.goodchoice.kr/resize_1000X500x0/...,서울 강남구 역삼동 677-14,['https://image.goodchoice.kr/resize_490x348/a...,"<article class=""detail_info"">\n<!-- 기본 정보 -->\..."


In [17]:
acc_detail = acc_detail.rename(columns={'Unnamed: 0':'id'})

In [18]:
acc_detail.head()

,id,category,name,score,region,ano,lat,lng,pic_url,address,pic_urls,detail
0,0,1,강남 더 뮤즈,9.8,강남구 역삼동,1824,37.502530,127.038275,https://image.goodchoice.kr/resize_1000X500x0/...,서울 강남구 역삼동 678-9,['https://image.goodchoice.kr/resize_490x348/a...,"<article class=""detail_info"">\n<!-- 기본 정보 -->\..."
1,1,1,역삼 호텔 디 아티스트,9.3,강남구 역삼동,5675,37.502415,127.038488,https://image.goodchoice.kr/resize_1000X500x0/...,서울 강남구 역삼동 678-13,['https://image.goodchoice.kr/resize_490x348/a...,"<article class=""detail_info"">\n<!-- 기본 정보 -->\..."
2,2,1,역삼 리치웰,9.3,강남구 역삼동,3805,37.502973,127.039823,https://image.goodchoice.kr/resize_1000X500x0/...,서울 강남구 역삼동 677-9,['https://image.goodchoice.kr/resize_490x348/a...,"<article class=""detail_info"">\n<!-- 기본 정보 -->\..."
3,3,1,강남 캠퍼스,9.3,강남구 역삼동,4988,37.497310,127.029190,https://image.goodchoice.kr/resize_1000X500x0/...,서울 강남구 역삼동 825-30,['https://image.goodchoice.kr/resize_490x348/a...,"<article class=""detail_info"">\n<!-- 기본 정보 -->\..."
4,4,1,역삼 인트로호텔,9.9,강남구 역삼동,72748,37.502754,127.039923,https://image.goodchoice.kr/resize_1000X500x0/...,서울 강남구 역삼동 677-14,['https://image.goodchoice.kr/resize_490x348/a...,"<article class=""detail_info"">\n<!-- 기본 정보 -->\..."


방 상세정보에 accommodationId 를 ano -> 숙소 id 로 변환

In [19]:
room_detail = pd.read_csv(f'../data/여기어때_방상세정보전체.csv')
room_detail = room_detail.rename(columns={'Unnamed: 0' : "roomId"})
room_detail.head()

,roomId,accommodationId,name,checkIn,checkOut,defaultPeople,maxPeople,roomCnt
0,0,1824,랜덤객실,23.0,12,2,2,1
1,1,1824,로이,23.0,12,2,2,1
2,2,1824,샴페인골드,23.0,12,2,2,1
3,3,1824,미스트,23.0,12,2,2,1
4,4,1824,에메랄드,23.0,12,2,2,1


In [20]:
for idx in tqdm(range(len(acc_detail))):
    cur = acc_detail.iloc[idx]
    ano = cur["ano"]
    id = cur["id"]

    room_detail = room_detail.replace({"accommodationId":ano}, id)

100%|██████████| 266/266 [00:00<00:00, 7566.53it/s]


In [48]:
# room table checkout 시간 1110 -> 11로 변경

room_detail = room_detail.replace({"checkOut": 1110}, 11)

In [49]:
room_detail.head()

,roomId,accommodationId,name,checkIn,checkOut,defaultPeople,maxPeople,roomCnt
0,0,0,랜덤객실,23.0,12,2,2,1
1,1,0,로이,23.0,12,2,2,1
2,2,0,샴페인골드,23.0,12,2,2,1
3,3,0,미스트,23.0,12,2,2,1
4,4,0,에메랄드,23.0,12,2,2,1


가격 테이블의 accommodationId 값 ano -> id 값으로 변경

In [43]:
price_detail = pd.read_csv(f'../data/여기어때_가격정보전체.csv')
price_detail = price_detail.rename(columns={'Unnamed: 0' : "priceId"})
price_detail.head()

,priceId,roomId,accommodationId,date,price,roomCnt,booked
0,0,0,1824,2023-07-01,40000.0,1,False
1,1,1,1824,2023-07-01,40000.0,1,False
2,2,2,1824,2023-07-01,40000.0,1,False
3,3,3,1824,2023-07-01,40000.0,1,False
4,4,4,1824,2023-07-01,40000.0,1,False


In [44]:
for idx in tqdm(range(len(acc_detail))):
    cur = acc_detail.iloc[idx]
    ano = cur["ano"]
    id = cur["id"]

    price_detail = price_detail.replace({"accommodationId":ano}, id)

100%|██████████| 266/266 [00:00<00:00, 781.22it/s]


In [45]:
price_detail = price_detail.iloc[:,:6]
price_detail.head()

,priceId,roomId,accommodationId,date,price,roomCnt
0,0,0,0,2023-07-01,40000.0,1
1,1,1,0,2023-07-01,40000.0,1
2,2,2,0,2023-07-01,40000.0,1
3,3,3,0,2023-07-01,40000.0,1
4,4,4,0,2023-07-01,40000.0,1


변경 값 저장

In [22]:
acc_detail.to_csv(f"../data/여기어때_숙소상세정보전체.csv" ,encoding="utf-8")
acc_detail.head(3)

,id,category,name,score,region,ano,lat,lng,pic_url,address,pic_urls,detail
0,0,1,강남 더 뮤즈,9.8,강남구 역삼동,1824,37.502530,127.038275,https://image.goodchoice.kr/resize_1000X500x0/...,서울 강남구 역삼동 678-9,['https://image.goodchoice.kr/resize_490x348/a...,"<article class=""detail_info"">\n<!-- 기본 정보 -->\..."
1,1,1,역삼 호텔 디 아티스트,9.3,강남구 역삼동,5675,37.502415,127.038488,https://image.goodchoice.kr/resize_1000X500x0/...,서울 강남구 역삼동 678-13,['https://image.goodchoice.kr/resize_490x348/a...,"<article class=""detail_info"">\n<!-- 기본 정보 -->\..."
2,2,1,역삼 리치웰,9.3,강남구 역삼동,3805,37.502973,127.039823,https://image.goodchoice.kr/resize_1000X500x0/...,서울 강남구 역삼동 677-9,['https://image.goodchoice.kr/resize_490x348/a...,"<article class=""detail_info"">\n<!-- 기본 정보 -->\..."


In [50]:
room_detail.to_csv(f"../data/여기어때_방상세정보전체.csv" ,encoding="utf-8")
room_detail.head(3)

,roomId,accommodationId,name,checkIn,checkOut,defaultPeople,maxPeople,roomCnt
0,0,0,랜덤객실,23.0,12,2,2,1
1,1,0,로이,23.0,12,2,2,1
2,2,0,샴페인골드,23.0,12,2,2,1


In [46]:
price_detail.to_csv(f"../data/여기어때_가격정보전체.csv" ,encoding="utf-8")
price_detail.head(3)

,priceId,roomId,accommodationId,date,price,roomCnt
0,0,0,0,2023-07-01,40000.0,1
1,1,1,0,2023-07-01,40000.0,1
2,2,2,0,2023-07-01,40000.0,1


가격정보 가격 결측치 수정

In [96]:
price_detail = pd.read_csv(f'../data/여기어때_가격정보전체.csv',encoding="utf-8")
price_detail.head()

,Unnamed: 0,priceId,roomId,accommodationId,date,price,roomCnt
0,0,0,0,0,2023-07-01,40000.0,1
1,1,1,1,0,2023-07-01,40000.0,1
2,2,2,2,0,2023-07-01,40000.0,1
3,3,3,3,0,2023-07-01,40000.0,1
4,4,4,4,0,2023-07-01,40000.0,1


In [97]:
# 결측치 확인
price_null = price_detail[price_detail['price'].isnull()]
len(price_null)

14653

In [109]:
# 결측치 중간 값으로 설정
price_detail["price"].fillna(price_detail.groupby("roomId")['price'].transform("median"), inplace=True)

/var/folders/tm/0x41_zqn6479q5jkctn74b6r0000gn/T/ipykernel_58592/1773363391.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  price_detail["price"].fillna(price_detail.groupby("roomId")['price'].transform("median"), inplace=True)


In [103]:
# 중복 값 제거
price_detail = price_detail.drop_duplicates(['roomId', 'accommodationId', 'price', 'date', 'roomCnt'], ignore_index=True)

In [110]:
price_detail

,Unnamed: 0,priceId,roomId,accommodationId,date,price,roomCnt
0,0,0,0,0,2023-07-01,40000.0,1
1,1,1,1,0,2023-07-01,40000.0,1
2,2,2,2,0,2023-07-01,40000.0,1
3,3,3,3,0,2023-07-01,40000.0,1
4,4,4,4,0,2023-07-01,40000.0,1
...,...,...,...,...,...,...,...
82092,112349,112349,1274,263,2023-09-29,250000.0,1
82093,112350,112350,1275,263,2023-09-29,650000.0,1
82094,112351,112351,1273,263,2023-09-30,170000.0,1
82095,112352,112352,1274,263,2023-09-30,250000.0,1


In [111]:
price_detail[(price_detail['price'].isnull())]

,Unnamed: 0,priceId,roomId,accommodationId,date,price,roomCnt
920,920,920,10,1,2023-07-01,NaN,1
921,921,921,11,1,2023-07-01,NaN,1
928,928,928,10,1,2023-07-02,NaN,1
929,929,929,11,1,2023-07-02,NaN,1
936,936,936,10,1,2023-07-03,NaN,1
...,...,...,...,...,...,...,...
31342,58687,58687,637,89,2023-09-28,NaN,1
31345,58690,58690,636,89,2023-09-29,NaN,1
31346,58691,58691,637,89,2023-09-29,NaN,1
31349,58694,58694,636,89,2023-09-30,NaN,1


In [116]:
new_price = price_detail.iloc[:,2:]
new_price.to_csv("../data/여기어때_가격정보전체수정.csv",encoding="utf-8")
new_price

,roomId,accommodationId,date,price,roomCnt
0,0,0,2023-07-01,40000.0,1
1,1,0,2023-07-01,40000.0,1
2,2,0,2023-07-01,40000.0,1
3,3,0,2023-07-01,40000.0,1
4,4,0,2023-07-01,40000.0,1
...,...,...,...,...,...
82092,1274,263,2023-09-29,250000.0,1
82093,1275,263,2023-09-29,650000.0,1
82094,1273,263,2023-09-30,170000.0,1
82095,1274,263,2023-09-30,250000.0,1


### 숙소 상세정보 중복 데이터 확인

database에 accommodation 데이터 삽입 시 primary key 중복으로 오류발생

중복된 값은 없었으나 id가 0부터 잡혀 있는데 1부터 들어갈 수 있어서 오류 발생
 1. 모든 id 값을 + 1 해주기
 2. id가 0인 값 제거 : 모든 테이블의 index가 0 인 값을 제거해줘야한다.
 3. id가 0인 값을 제일 큰 값으로 변경

모든 id 값 +1 해주기
숙소상세정보 수정

In [143]:
accommodation = pd.read_csv(f'../data/여기어때_숙소상세정보전체.csv')
accommodation.head()

,Unnamed: 0,id,category,name,score,region,ano,lat,lng,pic_url,address,pic_urls,detail
0,0,0,1,강남 더 뮤즈,9.8,강남구 역삼동,1824,37.502530,127.038275,https://image.goodchoice.kr/resize_1000X500x0/...,서울 강남구 역삼동 678-9,['https://image.goodchoice.kr/resize_490x348/a...,"<article class=""detail_info"">\n<!-- 기본 정보 -->\..."
1,1,1,1,역삼 호텔 디 아티스트,9.3,강남구 역삼동,5675,37.502415,127.038488,https://image.goodchoice.kr/resize_1000X500x0/...,서울 강남구 역삼동 678-13,['https://image.goodchoice.kr/resize_490x348/a...,"<article class=""detail_info"">\n<!-- 기본 정보 -->\..."
2,2,2,1,역삼 리치웰,9.3,강남구 역삼동,3805,37.502973,127.039823,https://image.goodchoice.kr/resize_1000X500x0/...,서울 강남구 역삼동 677-9,['https://image.goodchoice.kr/resize_490x348/a...,"<article class=""detail_info"">\n<!-- 기본 정보 -->\..."
3,3,3,1,강남 캠퍼스,9.3,강남구 역삼동,4988,37.497310,127.029190,https://image.goodchoice.kr/resize_1000X500x0/...,서울 강남구 역삼동 825-30,['https://image.goodchoice.kr/resize_490x348/a...,"<article class=""detail_info"">\n<!-- 기본 정보 -->\..."
4,4,4,1,역삼 인트로호텔,9.9,강남구 역삼동,72748,37.502754,127.039923,https://image.goodchoice.kr/resize_1000X500x0/...,서울 강남구 역삼동 677-14,['https://image.goodchoice.kr/resize_490x348/a...,"<article class=""detail_info"">\n<!-- 기본 정보 -->\..."


In [144]:
accommodation["id"] = accommodation["id"] + 1
accommodation = accommodation.iloc[:,1:]
accommodation.head()

,id,category,name,score,region,ano,lat,lng,pic_url,address,pic_urls,detail
0,1,1,강남 더 뮤즈,9.8,강남구 역삼동,1824,37.502530,127.038275,https://image.goodchoice.kr/resize_1000X500x0/...,서울 강남구 역삼동 678-9,['https://image.goodchoice.kr/resize_490x348/a...,"<article class=""detail_info"">\n<!-- 기본 정보 -->\..."
1,2,1,역삼 호텔 디 아티스트,9.3,강남구 역삼동,5675,37.502415,127.038488,https://image.goodchoice.kr/resize_1000X500x0/...,서울 강남구 역삼동 678-13,['https://image.goodchoice.kr/resize_490x348/a...,"<article class=""detail_info"">\n<!-- 기본 정보 -->\..."
2,3,1,역삼 리치웰,9.3,강남구 역삼동,3805,37.502973,127.039823,https://image.goodchoice.kr/resize_1000X500x0/...,서울 강남구 역삼동 677-9,['https://image.goodchoice.kr/resize_490x348/a...,"<article class=""detail_info"">\n<!-- 기본 정보 -->\..."
3,4,1,강남 캠퍼스,9.3,강남구 역삼동,4988,37.497310,127.029190,https://image.goodchoice.kr/resize_1000X500x0/...,서울 강남구 역삼동 825-30,['https://image.goodchoice.kr/resize_490x348/a...,"<article class=""detail_info"">\n<!-- 기본 정보 -->\..."
4,5,1,역삼 인트로호텔,9.9,강남구 역삼동,72748,37.502754,127.039923,https://image.goodchoice.kr/resize_1000X500x0/...,서울 강남구 역삼동 677-14,['https://image.goodchoice.kr/resize_490x348/a...,"<article class=""detail_info"">\n<!-- 기본 정보 -->\..."


In [145]:
accommodation.to_csv("../data/숙소상세정보.csv",encoding="utf-8")


방 상세정보 수정

In [150]:
room = pd.read_csv(f'../data/여기어때_방상세정보전체.csv')
room = room.iloc()[:,1:8]
room.head()

,roomId,accommodationId,name,checkIn,checkOut,defaultPeople,maxPeople
0,0,0,랜덤객실,23.0,12,2,2
1,1,0,로이,23.0,12,2,2
2,2,0,샴페인골드,23.0,12,2,2
3,3,0,미스트,23.0,12,2,2
4,4,0,에메랄드,23.0,12,2,2


In [151]:
room["roomId"] = room["roomId"] + 1
room["accommodationId"] = room["accommodationId"] + 1
room.head()

,roomId,accommodationId,name,checkIn,checkOut,defaultPeople,maxPeople
0,1,1,랜덤객실,23.0,12,2,2
1,2,1,로이,23.0,12,2,2
2,3,1,샴페인골드,23.0,12,2,2
3,4,1,미스트,23.0,12,2,2
4,5,1,에메랄드,23.0,12,2,2


In [152]:
room.to_csv("../data/방상세정보.csv",encoding="utf-8")

가격 정보 수정

In [171]:
price = pd.read_csv(f'../data/여기어때_가격정보전체수정.csv')
price = price.rename(columns={'Unnamed: 0' : "priceId"})
price.head()

,priceId,roomId,accommodationId,date,price,roomCnt
0,0,0,0,2023-07-01,40000.0,1
1,1,1,0,2023-07-01,40000.0,1
2,2,2,0,2023-07-01,40000.0,1
3,3,3,0,2023-07-01,40000.0,1
4,4,4,0,2023-07-01,40000.0,1


In [172]:
price['priceId'] = price['priceId'] + 1
price['roomId'] = price['roomId'] + 1
price['accommodationId'] = price['accommodationId'] + 1
price.head()

,priceId,roomId,accommodationId,date,price,roomCnt
0,1,1,1,2023-07-01,40000.0,1
1,2,2,1,2023-07-01,40000.0,1
2,3,3,1,2023-07-01,40000.0,1
3,4,4,1,2023-07-01,40000.0,1
4,5,5,1,2023-07-01,40000.0,1


In [173]:
price.to_csv("../data/가격정보.csv",encoding="utf-8")


사진 정보 수정

In [174]:
picture = pd.read_csv(f'../data/숙소사진.csv')
picture = picture.iloc()[:,1:8]
picture.head()

,pictureId,accommodationId,url
0,0,0,https://image.goodchoice.kr/resize_490x348/adi...
1,1,0,https://image.goodchoice.kr/resize_490x348/adi...
2,2,0,https://image.goodchoice.kr/resize_490x348/adi...
3,3,0,https://image.goodchoice.kr/resize_490x348/adi...
4,4,0,https://image.goodchoice.kr/resize_490x348/adi...


In [175]:
picture.pictureId = picture.pictureId + 1
picture.accommodationId = picture.accommodationId + 1
picture.head()

,pictureId,accommodationId,url
0,1,1,https://image.goodchoice.kr/resize_490x348/adi...
1,2,1,https://image.goodchoice.kr/resize_490x348/adi...
2,3,1,https://image.goodchoice.kr/resize_490x348/adi...
3,4,1,https://image.goodchoice.kr/resize_490x348/adi...
4,5,1,https://image.goodchoice.kr/resize_490x348/adi...


In [176]:
picture.to_csv("../data/숙소사진.csv",encoding="utf-8")